In [ ]:
#| default_exp core

# core

In [ ]:
#|export
import os,shutil,subprocess,tempfile
from contextlib import contextmanager
from pathlib import Path

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export
class TmpDir:
    "Create temporary workspaces."
    def __init__(self): self.cwd,self._dir,self.path = None,None,None

    def new(self, subdir=''):
        "Create and `cd` to `subdir` under a temp dir."
        if self.cwd is None: self.cwd = Path.cwd()
        self.dir = Path(tempfile.mkdtemp())
        self.path = self.dir/subdir
        self.path.mkdir(exist_ok=True, parents=True)
        os.chdir(self.path)

    @property
    def dir(self): return self._dir
    @dir.setter
    def dir(self, o):
        "`rm` current `dir` and set a new one."
        if self._dir: shutil.rmtree(self._dir)
        self._dir = o

    def close(self):
        "`rm` current `dir` and `cd` to original `cwd`."
        self.dir = None
        os.chdir(self.cwd)

    @contextmanager
    def __call__(self, subdir=''):
        "Work in a temp dir then `cd` back to original `cwd`."
        try:
            self.new(subdir)
            yield self.path
        finally: self.close()

The main way to use `TmpDir` is through the exported object:

In [ ]:
#|exports
tmpdir = TmpDir()

We start out in our project dir:

In [ ]:
d0 = Path.cwd(); d0

Path('/Users/seem/code/sketch-tmpdir')

Switch to path `foo/bar` under a temp dir:

In [ ]:
tmpdir.new('foo/bar')
d1 = tmpdir.dir
Path.cwd()

Path('/private/var/folders/ft/0gnvc3ts5jz4ddqtttp6tjvm0000gn/T/tmp19y8l0h4/foo/bar')

If we switch again, the previous dir is removed:

In [ ]:
tmpdir.new('foo/bar')
assert not d1.exists()
d2 = tmpdir.dir
Path.cwd()

Path('/private/var/folders/ft/0gnvc3ts5jz4ddqtttp6tjvm0000gn/T/tmpnjqprlft/foo/bar')

Finally, revert to the original working directory, which also removes the remaining temporary directory:

In [ ]:
tmpdir.close()
assert not d2.exists()
test_eq(Path.cwd(), d0)

You can also use it as a context manager to automatically revert to the original working directory at the end:

In [ ]:
with tmpdir() as p:
    d3 = tmpdir.dir
    test_eq(Path.cwd().name, p.name)
assert not d3.exists()
test_eq(Path.cwd(), d0)

The primary use-case is to write executable documentation for code that interacts with its workspace, as described in the examples below.

## Example 1: Python git interface

In [ ]:
def git_repo():
    "Remote repo from git config."
    cmd = 'git config --get remote.origin.url'
    proc = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if proc.returncode: return
    return proc.stdout.strip().split('/', 1)[1].split('.')[0]

Let's initialise a minimal repo to demonstrate:

In [ ]:
tmpdir.new()

In [ ]:
%%sh
git init -q
git remote add origin git@github.com:my-user/my-repo.git

Get the repo name:

In [ ]:
test_eq(git_repo(), 'my-repo')

Returns `None` if you're not in a git repo:

In [ ]:
with tmpdir(): test_is(git_repo(), None)

And you're back to your original working directory:

In [ ]:
Path.cwd()

Path('/Users/seem/code/sketch-tmpdir')

## Example 2: nbdev hooks

This section shows what the [end-to-end nbdev hooks test](https://github.com/fastai/nbdev/blob/master/nbs/10_clean.ipynb) could look like with `tmpdir`. The test checks that nbdev's notebook-aware merge driver works. In order to do that we need to simulate a merge conflict, which involves some back-and-forth with git.

In [ ]:
from copy import deepcopy
from execnb.nbio import dict2nb, mk_cell, read_nb, write_nb
from fastcore.foundation import Config
from nbdev.read import create_output, show_src

Start a new workspace:

In [ ]:
tmpdir.new()

Init a git repo and checkout our main branch -- this is where we'll run our tests:

In [ ]:
%%sh
git init

Initialized empty Git repository in /private/var/folders/ft/0gnvc3ts5jz4ddqtttp6tjvm0000gn/T/tmpggvpu_sy/.git/


In [ ]:
%%sh
git checkout -b main

Switched to a new branch 'main'


Install nbdev hooks:

In [ ]:
%%sh
nbdev_install_hooks

Hooks are installed.


Next, we'll simulate a merge conflict. First we add `random.ipynb` notebook to `main`. Here's what it looks like to start with:

In [ ]:
#|hide
from fastcore.utils import *

In [ ]:
#|hide
def show_nb(fn):
    nb = read_nb(fn)
    head = f'# {fn}'
    body = [head]
    for cell in nb.cells:
        c = f'# %%\n' + cell.source
        out = nested_idx(cell, 'output', 0, 'data', 'text/plain')
        if out: c += ''.join('\n# '+o.strip() for o in out)
        body.append(c)
    return show_src('\n\n'.join(body))

In [ ]:
fn = 'random.ipynb'
meta = {'nbformat': 4,'metadata':{'kernelspec':{'display_name':'Python 3','language': 'python','name': 'python3'}}}
base = dict2nb({'cells':[mk_cell('import random'), mk_cell('random.random()')], **meta})
base.cells[-1].output = create_output('0.3314001088639852\n0.20280244713400464', 'plain')
write_nb(base, fn)
show_nb(fn)

```python
# random.ipynb

# %%
import random

# %%
random.random()
# 0.3314001088639852
# 0.20280244713400464
```

Commit it:

In [ ]:
%%sh
git add .
git commit -q -m 'add random.ipynb'

Next, checkout a new branch `add-heading`:

In [ ]:
%%sh
git checkout -b add-heading

Switched to a new branch 'add-heading'


Make a change. We added a new markdown cell `Calculate a random number:`, imported `os`, and got different `random.random()` outputs -- the perfect recipe for a merge conflict:

In [ ]:
ours = deepcopy(base)
ours.cells[0].source+=',os' # Change first cell
ours.cells.insert(1, mk_cell('Calculate a random number:', cell_type='markdown')) # New cell
ours.cells[-1].output = create_output('0.3379097372590093\n0.7379492349993123', 'plain') # Change outputs
write_nb(ours, fn)
show_nb(fn)

```python
# random.ipynb

# %%
import random,os

# %%
Calculate a random number:

# %%
random.random()
# 0.3379097372590093
# 0.7379492349993123
```

Commit it:

In [ ]:
%%sh
git commit -am heading

[add-heading 5b71b21] heading
 1 file changed, 10 insertions(+), 3 deletions(-)


Go back to main:

In [ ]:
%%sh
git checkout main

Switched to branch 'main'


Make a different change:

In [ ]:
#|hide
thrs = deepcopy(base)
thrs.cells[0].source+=',sys'# Also change first cell
thrs.cells.insert(0, mk_cell('# Random numbers', cell_type='markdown')) # New cell
thrs.cells[-1].output = create_output('0.6587181429602441\n0.5962200692415515', 'plain') # Change outputs
write_nb(thrs, fn)

In [ ]:
show_nb(fn)

```python
# random.ipynb

# %%
# Random numbers

# %%
import random,sys

# %%
random.random()
# 0.6587181429602441
# 0.5962200692415515
```

Commit it:

In [ ]:
%%sh
git commit -am docs

[main d3b0252] docs
 1 file changed, 10 insertions(+), 3 deletions(-)


And finally try to merge:

In [ ]:
%%sh --no-raise-error
git merge add-heading

One or more conflict remains in the notebook, please inspect manually.
Auto-merging random.ipynb
CONFLICT (content): Merge conflict in random.ipynb
Automatic merge failed; fix conflicts and then commit the result.


We have a merge conflict! But thanks to nbdev:

1. Conflicting outputs are automatically resolved
2. The notebook is left in a readable state.

In [ ]:
show_nb(fn)

```python
# random.ipynb

# %%
# Random numbers

# %%
`<<<<<<< HEAD`

# %%
import random,sys

# %%
`=======`

# %%
import random,os

# %%
Calculate a random number:

# %%
`>>>>>>> add-heading`

# %%
random.random()
# 0.6587181429602441
# 0.5962200692415515
```

Close `tmpdir`, and you're back to your original working directory:

In [ ]:
tmpdir.close()
Path.cwd()

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()